In [15]:
import os 
import pandas

In [34]:
rootdir = input('Specify the directory with the data folder and config: ')
#rootdir = r"C:\Users\derek\OneDrive\Desktop\UCI\RESEARCH\HPLC Peak Picking"
rootdir = rootdir+'\\'
print(rootdir)

Datafolder = input('Specify the name of the data folder: ')
output = open(Datafolder+".csv", "w")
#21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46


Specify the directory with the data folder and config: 
C:\Users\derek\OneDrive\Desktop\UCI\RESEARCH\HPLC Peak Picking\
Specify the name of the data folder: 21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46


# This block reads the config file

In [35]:

def readconfig(path):
    config = open(path+'config.txt', 'r')
    DAD = []
    RID = []
    Key = ''
    for line in config:
        if line[0] == '>':
            line = line.strip('>')
            if line[0] == 'D':
                Key = "D"
            if line[0] == 'R':
                Key = 'R'
            if line[0] == 'T':
                Key = 'T'
                line = line.split('=')
                Tol = line[-1].strip()
            continue
        if Key == "D":
            if line.strip() != '':
                DAD.append(line.strip())
        if Key == 'R':
            if line.strip() != '':
                RID.append(line.strip())
    print(DAD,RID)
    return DAD, RID, Tol


# Unpack the config into DAD and RID retentions times and column labels

In [36]:
DAD, RID, Tol = readconfig(rootdir)
DAD_times = []
RID_times = []
for entry in DAD:
    DAD_times.append(float(entry.split()[-1]))
print(DAD_times)
for entry in RID:
    RID_times.append(float(entry.split()[-1]))
print(RID_times)

['Gluconate = 8.7', 'Pyruvate = 10.74'] ['Glucose&gluconate = 9', 'Pyruvate = 11']
[8.7, 10.74]
[9.0, 11.0]


# Peak Pick and export identified as Csv

In [37]:
%%time
titleline = 'Folder,datafolder,sample name,'
for entry in DAD:
    titleline += entry+'_DAD,'
for entry in RID:
    titleline += entry+'_RID,'
print(titleline)

output.write(titleline+'\n')
for subdir, dirs, files in os.walk(rootdir+'\\'+Datafolder):
    #print (subdir)
    for file in files:
        peaks = False
        key = ''
        row = {}
        rowr = {}
        if file == 'Report.TXT':
            for i in DAD_times:
                row[i] = 0
            for i in RID_times:
                rowr[i] = 0
            report = open(subdir+'\\'+file,'r',encoding='utf-16')
            for line in report:
                if 'Sample Name' in line:
                    samplename = line.strip().split(':')[-1]
                    print(samplename)
                if ': DAD' in line:
                    key = 'DAD'
                    continue
                if ': RID' in line:
                    key = 'RID'
                    continue
                if key == 'DAD':
                    line = line.strip().split()
                    #print(line)
                    if line == []:
                        continue
                    if line[0] == '1':
                        peaks = True
                    if line[0] == 'Totals':
                        key = ''
                        peaks = False
                        continue
                    if peaks:
                        for t,rt in enumerate(DAD_times):
                            if (float(line[1]) > (rt - float(Tol))) and (float(line[1]) < (rt + float(Tol))):
                                row[rt] = float(line[4])
                                #print(row)
                if key == 'RID':
                    line = line.strip().split()
                    #print(line)
                    if line == []:
                        continue
                    if line[0] == '1':
                        peaks = True
                    if line[0] == 'Totals':
                        key = ''
                        peaks = False
                    if peaks:
                        for t,rt in enumerate(RID_times):
                            if (float(line[1]) > (rt - float(Tol))) and (float(line[1]) < (rt + float(Tol))):
                                rowr[rt] = float(line[4])
                                #print(rowr)
            label = ','.join(subdir.split('\\')[-2:])+','+samplename 
            for t in DAD_times:
                label += ','+str(row[t])
            for t in RID_times:
                label += ','+str(rowr[t])
            label = label+'\n'
            print(label)
                #label  += row[t]
            output.write(label)
            print(label)
            report.close()
                #print (subdir, file)
                #output.write(','.join(subdir.split('\\')[-2:])+'\n')
                #print(','.join(subdir.split('\\')[-2:])+'\n')
output.close()

Folder,datafolder,sample name,Gluconate = 8.7_DAD,Pyruvate = 10.74_DAD,Glucose&gluconate = 9_RID,Pyruvate = 11_RID,
 dummy
21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46,001-0201.D, dummy,0,17775.3,3193300.0,452570.0

21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46,001-0201.D, dummy,0,17775.3,3193300.0,452570.0

 1
21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46,001-0301.D, 1,0,17769.8,3195030.0,451966.0

21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46,001-0301.D, 1,0,17769.8,3195030.0,451966.0

 2
21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46,002-0401.D, 2,0,17735.6,3192240.0,449956.0

21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46,002-0401.D, 2,0,17735.6,3192240.0,449956.0

 3
21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46,003-0501.D, 3,0,17760.0,3181940.0,451215.0

21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46,003-0501.D, 3,0,17760.0,3181940.0,451215.0

 4
21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46,004-0601.D, 4,0,17933.0,3072000.0,456

# Combine all reports into a single readable raw data text report

In [38]:
%%time
output2 = open(Datafolder+"_Raw.txt", "w")
for subdir, dirs, files in os.walk(rootdir+'\\'+Datafolder):
    for file in files:
        if file == 'Report.TXT':
            report = open(subdir+'\\'+file,'r',encoding='utf-16')
            for line in report:
                print(line)
                output2.write(line)
output2.close()
                

Data File C:\CHEM32\1\DATA\DEREKTEST\21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46\001-0201.D

Sample Name: dummy

Instrument 1 2/6/2021 11:23:31 AM Derek




Acq. Operator   : Derek                          Seq. Line :   2

Acq. Instrument : Instrument 1                    Location : Vial 1

Injection Date  : 2/4/2021 8:45:51 PM                  Inj :   1

                                                Inj Volume : 20.0 µl

Acq. Method     : C:\CHEM32\1\DATA\DEREKTEST\21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46\21_

                  02_04_GLUCOSE_GLUCONATE_INVITRO.M

Last changed    : 2/4/2021 12:46:21 PM by Derek

Analysis Method : C:\CHEM32\1\METHODS\DEREK TEST\PURGE.M

Last changed    : 1/27/2021 1:22:07 PM by Derek

Method Info     : BioRad Aminex HPX-87H

                  300 x 7.8mm, Isocratic, RID

                  20ul injection  100 bar Max Pressure Click for more info

                  Capillary ID: 0.17mm

                  Retention Times

              

Data File C:\CHEM32\1\DATA\DEREKTEST\21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46\029-3101.D

Sample Name: s1

Instrument 1 2/6/2021 1:56:26 PM Derek




Acq. Operator   : Derek                          Seq. Line :  31

Acq. Instrument : Instrument 1                    Location : Vial 29

Injection Date  : 2/5/2021 9:36:22 AM                  Inj :   1

                                                Inj Volume : 20.0 µl

Acq. Method     : C:\CHEM32\1\DATA\DEREKTEST\21_02_04_INVITRO_NAD_EXCLUSIONS 2021-02-04 20-18-46\21_

                  02_04_GLUCOSE_GLUCONATE_INVITRO.M

Last changed    : 2/4/2021 12:46:21 PM by Derek

Analysis Method : C:\CHEM32\1\METHODS\DEREK TEST\PURGE.M

Last changed    : 1/27/2021 1:22:07 PM by Derek

Method Info     : BioRad Aminex HPX-87H

                  300 x 7.8mm, Isocratic, RID

                  20ul injection  100 bar Max Pressure Click for more info

                  Capillary ID: 0.17mm

                  Retention Times

                 


   3   8.954 VB    0.4046 1.12512e4   419.91214  36.9449

   4  10.770 BB    0.2131 1.83070e4  1309.25720  60.1137

   5  13.771 BB    0.2941   89.41193    4.69636   0.2936

Totals :                  3.04540e4  1787.21543





Signal 2: RID1 A, Refractive Index Signal



Peak RetTime Type  Width     Area      Height     Area  

  #   [min]        [min]  [nRIU*s]    [nRIU]        %

----|-------|----|-------|----------|----------|--------|

   1   7.398 BB    0.2604 1093.59509   54.91365   0.0150

   2   8.558 BV    0.1517 4913.50391  508.23822   0.0672

   3   9.074 VB    0.2494 2.91707e6  1.80101e5   39.8866

   4  11.040 BB    0.2262 4.64605e5  3.14729e4    6.3528

   5  13.572 VB    0.2653 1.10224e6  6.34651e4   15.0714

   6  18.861 VB    0.4450 2.81731e6  9.56992e4   38.5225

   7  20.761 BV    0.1971  703.94397   43.09324 9.625e-3

   8  20.840 VV    0.1472  521.79974   42.60592 7.135e-3

   9  22.576 BV    0.4100 2174.80933   64.25118   0.0297

  10  23.314 VB    0.3696 1317.40